In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train/train.csv')

In [3]:
df.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df=df.dropna()

In [6]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
X=df.drop('label',axis=1)


In [8]:
y=df['label']

In [9]:
print(X.shape)
print(y.shape)

(18285, 4)
(18285,)


In [10]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [11]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
voc_size=5000
messages=X.copy()
messages.reset_index(inplace=True)

In [13]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print("Status: %s / %s" %(i, len(messages)), end="\r")
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [14]:
from tensorflow.keras.preprocessing.text import one_hot

In [15]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length = 20
embedded_doc=pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
print(embedded_doc)

[[   0    0    0 ... 3488  877  844]
 [   0    0    0 ...  886 3751 1864]
 [   0    0    0 ... 4959 2774 4244]
 ...
 [   0    0    0 ... 2897 2884 3524]
 [   0    0    0 ... 4943 2331 4956]
 [   0    0    0 ...  292 3247  656]]


In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input,GlobalMaxPool1D,Dropout


In [18]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
len(embedded_doc),y.shape

(18285, (18285,))

In [20]:
import numpy as np

In [21]:
X_final=np.array(embedded_doc)
y_final=np.array(y)
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [23]:
 model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=256)

Epoch 1/10
39/39 [==============================] - 4s 97ms/step - loss: 0.5982 - accuracy: 0.6946 - val_loss: 0.4522 - val_accuracy: 0.7984
Epoch 2/10
39/39 [==============================] - 3s 81ms/step - loss: 0.3226 - accuracy: 0.8613 - val_loss: 0.2865 - val_accuracy: 0.8722
Epoch 3/10
39/39 [==============================] - 3s 84ms/step - loss: 0.1868 - accuracy: 0.9238 - val_loss: 0.2368 - val_accuracy: 0.8914
Epoch 4/10
39/39 [==============================] - 3s 83ms/step - loss: 0.1276 - accuracy: 0.9527 - val_loss: 0.2303 - val_accuracy: 0.9024
Epoch 5/10
39/39 [==============================] - 3s 82ms/step - loss: 0.0917 - accuracy: 0.9654 - val_loss: 0.2617 - val_accuracy: 0.8857
Epoch 6/10
39/39 [==============================] - 3s 84ms/step - loss: 0.0775 - accuracy: 0.9752 - val_loss: 0.2830 - val_accuracy: 0.8931
Epoch 7/10
39/39 [==============================] - 3s 82ms/step - loss: 0.0563 - accuracy: 0.9812 - val_loss: 0.2516 - val_accuracy: 0.9118
Epoch 8/10
39

In [24]:
y_pred1=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)


array([[3152,  267],
       [ 321, 2295]], dtype=int64)

In [32]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred1)

In [33]:
accuracy

0.9025683512841757

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      3419
           1       0.90      0.88      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035



In [28]:
from keras.models import load_model

#model.save('model_sequence_title.h5')

In [29]:
import tensorflow as tf
print(tf.__version__)


2.3.1
